In [ ]:
# Install the necessary libraries (Only run once)
!pip install transformers

In [4]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

# Load the BERT tokenizer and model
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertForSequenceClassification.from_pretrained(model_name)

# Sample data
train_sentences = ["I love this product!", "It's terrible and broken."]
train_labels = [1, 0]  # 1: Positive, 0: Negative

# Convert texts to InputExamples
train_examples = [InputExample(guid=str(i), text_a=train_sentences[i], text_b=None, label=train_labels[i]) for i in range(len(train_sentences))]

# Convert InputExamples to InputFeatures
def convert_examples_to_features(examples, tokenizer, max_length=128, label_list=[0,1], output_mode="classification"):
    features = []
    for example in examples:
        inputs = tokenizer.encode_plus(
            example.text_a,
            example.text_b,
            max_length=max_length,
            pad_to_max_length=True,
            return_attention_mask=True
        )
        label = label_list.index(example.label)
        features.append(InputFeatures(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], label=label))
    return features

train_features = convert_examples_to_features(train_examples, tokenizer)
print(train_features[0])

# Convert InputFeatures to TensorFlow datasets
def convert_features_to_tf_dataset(features):
    all_input_ids = []
    all_attention_masks = []
    all_labels = []

    for feature in features:
        all_input_ids.append(feature.input_ids)
        all_attention_masks.append(feature.attention_mask)
        all_labels.append(feature.label)

    print(all_labels)
    return tf.data.Dataset.from_tensor_slices(({"input_ids": all_input_ids, "attention_mask": all_attention_masks}, all_labels))

train_dataset = convert_features_to_tf_dataset(train_features)


# Train the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_dataset.shuffle(100).batch(32), epochs=3, batch_size=32)

# Predict
test_sentence = "I think this is amazing!"
test_input = tokenizer.encode_plus(test_sentence, add_special_tokens=True, return_tensors="tf")
outputs = model(test_input["input_ids"], attention_mask=test_input["attention_mask"])
probs = tf.nn.softmax(outputs[0], axis=-1)
predicted_label = tf.argmax(probs, axis=1).numpy()[0]
print(f"Predicted label: {predicted_label}")


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


InputFeatures(input_ids=[101, 1045, 2293, 2023, 4031, 999, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=None, label=1)
[1, 0]
Epoch 1/3


/usr/local/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:2622: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


1/1 [==============================] - 49s 49s/step - loss: 0.6961 - accuracy: 0.5000
Epoch 2/3
1/1 [==============================] - 4s 4s/step - loss: 0.5556 - accuracy: 1.0000
Epoch 3/3
1/1 [==============================] - 4s 4s/step - loss: 0.6738 - accuracy: 0.5000
Predicted label: 1


In [7]:
print("train ds type",type(train_dataset))

train ds type <class 'tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset'>
